In [ ]:
import torch
from transformers import ViTImageProcessor, ViTForImageClassification, TrainingArguments

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print(torch.cuda.get_device_name(0))
    print(torch.cuda.get_device_properties(0))
else:
    device = torch.device("cpu")
    print("using CPU, GPU is not available")

In [ ]:
model_name = "google/vit-base-patch16-224"
image_processor = ViTImageProcessor.from_pretrained(model_name)
model = ViTForImageClassification.from_pretrained(model_name)

**Load Dataset**

In [ ]:
from dataset import GetMedMNIST

d_size = 224

train_dataloader = GetMedMNIST(name='BloodMNIST', split='train', size=d_size)
valid_dataloader = GetMedMNIST(name='BloodMNIST', split='val', size=d_size, verbose=False)
test_dataloader = GetMedMNIST(name='BloodMNIST', split='train', size=d_size, verbose=False)

In [ ]:
model = ViTForImageClassification.from_pretrained(
    model_name,
    num_labels=train_dataloader.n_classes,
    id2label={str(i): c for i, c in enumerate(train_dataloader.label_str)},
    label2id={c: str(i) for i, c in enumerate(train_dataloader.label_str)},
    ignore_mismatched_sizes=True,
)

In [ ]:
training_args = TrainingArguments(
  output_dir="./BloodMNIST_ViT",
  per_device_train_batch_size=32,
  evaluation_strategy="steps",
  num_train_epochs=10,
  fp16=True,
  save_steps=374,
  eval_steps=374,
  logging_steps=374,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
)


In [ ]:
from evaluate import load
import numpy as np

accuracy = load("accuracy")
f1 = load("f1")

def compute_metrics(eval_pred):
  accuracy_score = accuracy.compute(predictions=np.argmax(eval_pred.predictions, axis=1), references=eval_pred.label_ids)
  f1_score = f1.compute(predictions=np.argmax(eval_pred.predictions, axis=1), references=eval_pred.label_ids, average="weighted")
  return {**accuracy_score, **f1_score}
     

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataloader,
    eval_dataset=valid_dataloader,
    tokenizer=image_processor
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate(test_dataloader)

In [ ]:
outputs = trainer.predict(test_dataloader)
y_true = outputs.label_ids
y_pred = outputs.predictions.argmax(1)

**Evaluation with Confusion Matrix**

**Generate Precision, Recall, F1 Score, Support**

**Generate Training & Loss Curves**

In [ ]:
import matplotlib.pyplot as plt

data = trainer.state.log_history
epochs = [entry['epoch'] for entry in data]
train_losses = [entry['loss'] for entry in data if 'loss' in entry]
eval_losses = [entry['eval_loss'] for entry in data if 'eval_loss' in entry]
plt.figure(figsize=(6, 4))

plt.plot(epochs[:len(train_losses)], train_losses, label='Training Loss')
plt.plot(epochs[:len(eval_losses)], eval_losses, label='Validation Loss')

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

**Plot ROC Curves**

In [ ]:
from sklearn.metrics import roc_curve, auc, roc_auc_score
import matplotlib.pyplot as plt
import numpy as np

true_labels = outputs.label_ids
predictions = outputs.predictions

roc_score = roc_auc_score(true_labels, predictions)

num_classes = predictions.shape[1]
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(num_classes):
    fpr[i], tpr[i], _ = roc_curve((true_labels == i).astype(int), predictions[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

plt.figure(figsize=(6, 4))

for i in range(num_classes):
    plt.plot(fpr[i], tpr[i], lw=2, label='ROC curve (class %d) (AUC = %0.5f)' % (i, roc_auc[i]))

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")
plt.show()